# Make_PathInfo

In [ ]:
#make_pathInfo_1

from modules.dataset import *
from modules.patch import *
from modules.slide_info import *

import openslide, argparse, pickle, os
import numpy as np


def main():
   
    parser = argparse.ArgumentParser(description="variable file select")
    parser.add_argument("--tsv", default='codipai.tsv', type=str, help="variable tsv file")
    args, unknown = parser.parse_known_args()
    variableFile = args.tsv

    print("Import file")
    inputValue = {}
    for key, value in np.loadtxt(variableFile, dtype=str, delimiter='\t'):
        inputValue[key] = value
    print("Complete\n")

    print("Match file path")
    pathRootWSIList = []
    pathRootAnnoList = []
    for key, value in inputValue.items():
        value = value.split("\n")[0]
        if "pathRootWSI" in key:
            pathRootWSIList.append(value)
        if "pathRootAnno" in key:
            pathRootAnnoList.append(value)

    pathWSIDic = {}
    for pathRootWSI in pathRootWSIList:
        pathWSIDic = {**pathWSIDic, **ReadFilesSubDir(pathRootWSI, [".svs", ".tiff"])}

    pathAnnoDic = {}
    for pathRootAnno in pathRootAnnoList:
        pathAnnoDic = {**pathAnnoDic, **ReadFilesSubDir(pathRootAnno, [".xml", ".geojson"])}

    intersectID = np.intersect1d(list(pathWSIDic.keys()), list(pathAnnoDic.keys()))
    pathDicSum = {}
    for ID in intersectID:
        pathDicSum[ID] = {"pathWSI" : pathWSIDic[ID], "pathAnno" : pathAnnoDic[ID]}
    print("Complete\n")

    print("Save path pack")
    pickle.dump(pathDicSum, open("./sourcecode/CAMELYON16/pathCaMWSINAnno_464.dump", "wb"))
    print("Complete\n")
    
    print("Integrity check")
    errorSlide = []
    for ID in pathWSIDic:
        try:
            handleWSI = openslide.OpenSlide(pathWSIDic[ID])
            boundDic = GetBoundDic(handleWSI)
            img = GetWSIThumbnail(handleWSI, handleWSI.level_downsamples[-1], boundDic)
        except:
            print("error in slide %s"%pathWSIDic[ID])
            errorSlide.append(pathWSIDic[ID])
            np.savetxt("./data/PATH/errorSlide_2.txt", errorSlide, fmt="%s")
    print("Complete\n")
    
    return None

if __name__ == "__main__":
    main()

# Extracts patches from WSI images

In [ ]:
from cam_dataloader import *
import pickle

magnification=0.625
scale = get_scale_by_magnification(magnification)
tile_size=20
tile_size_original = int(scale * tile_size)


pathDicSum = pickle.load(open('./sourcecode/CAMELYON16/pathCaMWSINAnno_464.dump', "rb"))
for ID in pathDicSum :    
    wsi_images_dir = pathDicSum[ID]["pathWSI"]
    annotations_dir = pathDicSum[ID]["pathAnno"]
    patch_images_dir = './sourcecode/CAMELYON16/save_dir/dataset_2'
    save_patches = './sourcecode/CAMELYON16/save_dir/dataset_2/patches'

    wsi_image_file = wsi_images_dir
    wsi_image_number = ID

    logger.info("Processing wsi '{}'".format(ID))

    # check directory to save image-patches
    dir_to_save = "{}".format(save_patches)
    dir_to_save_fol = "{}/{}".format(patch_images_dir, wsi_image_number)

    if not os.path.exists(dir_to_save):
        os.makedirs("{}/01-original".format(dir_to_save))
        os.makedirs("{}/02-mask".format(dir_to_save))
        os.makedirs("{}/03-roi".format(dir_to_save))
    if not os.path.exists(dir_to_save_fol):
        os.makedirs("{}".format(dir_to_save_fol))

    contours = ParserQuPathGeojson(annotations_dir)

    # tumor tissue region
    np_scaled_down_image, np_regions_label, np_tumor_mask, np_tumor_masked = extract_tumor_region_from_wsi(contours, wsi_image_file, magnification)
    pil_scaled_down_image = np_to_pil(np_scaled_down_image)
    pil_tumor_mask = np_to_pil(np_tumor_mask)
    pil_tumor_masked = np_to_pil(np_tumor_masked)

    # normal tissue region 
    np_normal_mask, np_normal_masked = extract_normal_region_from_wsi(wsi_image_file, np_scaled_down_image, np_tumor_mask)
    pil_normal_mask = np_to_pil(np_normal_mask)
    pil_normal_masked = np_to_pil(np_normal_masked)
    pil_background = np_to_pil(np_normal_mask | np_tumor_mask)

    blend_tumor = blend_image(pil_scaled_down_image, pil_tumor_mask, foreground='red', inverse=True)
    blend_tissue = blend_image(blend_tumor, pil_normal_mask, foreground='green', inverse=True)
    blend_background = blend_image(blend_tissue, pil_background, foreground='blue', inverse=False)

    pil_scaled_down_image.save('{}/{}_1.png'.format(dir_to_save_fol, wsi_image_number))
    blend_background.save('{}/{}_2.png'.format(dir_to_save_fol, wsi_image_number))
    pil_normal_masked.save('{}/{}_3.png'.format(dir_to_save_fol, wsi_image_number))
    pil_tumor_masked.save('{}/{}_4.png'.format(dir_to_save_fol, wsi_image_number))

    #heat grid normal
    pil_img_normal_result, heat_grid_normal, number_of_tiles_normal = draw_heat_grid(np_normal_masked, tile_size)
    pil_img_normal_result.save('{}/{}_5.png'.format(dir_to_save_fol, wsi_image_number))

    # extract and save normal image-patches
    count_tiles_normal = 0
    for idx, (position, row, column, location, size, color) in enumerate(heat_grid_normal):
        if color != GREEN_COLOR:                    
            count_tiles_normal += 1
        ##############

    # heat grid tumor
    pil_img_result, heat_grid, number_of_tiles = draw_heat_grid(np_tumor_masked, tile_size)
    pil_img_result.save('{}/{}_6.png'.format(dir_to_save_fol, wsi_image_number))

    # extract and save tumor imag-patches
    count_tiles_tumor = 0
    for idx, (position, row, column, location, size, color) in enumerate(heat_grid):

        if color != GREEN_COLOR:  # color == RED_COLOR or color == ORANGE_COLOR:

            r_s = row * tile_size
            r_e = r_s + tile_size
            c_s = column * tile_size
            c_e = c_s + tile_size
            np_tile_masked = np_tumor_masked[r_s:r_e, c_s:c_e]
            np_tile_mask = np_tumor_mask[r_s:r_e, c_s:c_e]

            # only tile with valid size
            if np_tile_masked.shape[0] == tile_size and np_tile_masked.shape[1] == tile_size:

                tile_pil, tile_np = read_region(wsi_image_file, column, row, magnification, tile_size)
                left = (column * tile_size_original)
                top = (row * tile_size_original)

                pil_mask = np_to_pil(np.zeros((tile_np.shape[0], tile_np.shape[1]), dtype=np.uint8))
                draw = ImageDraw.Draw(pil_mask)
                for idx, (points) in enumerate(contours):
#                         if group != "_2" and group != "Exclusion" and len(points) > 1:
                    points_scaled = [(p[0]-left, p[1]-top) for p in points]
                    draw.polygon(points_scaled, outline=None, fill=1)

                np_tile_mask = pil_to_np(pil_mask).astype(bool)
                np_tile_masked = mask_rgb(tile_np, np_tile_mask)

                pil_tile_roi = blend_image(np_to_pil(tile_np), np_to_pil(np_tile_mask), foreground='red', inverse=True)
                #pil_tile_roi = blend_image(pil_tile_roi, np_to_pil(np_tile_mask), foreground='green', inverse=False)

                # save the extracted tumor image-patch
                tile_pil.save('{}/{}/{}_r{}c{}.png'.format(dir_to_save, "01-original", wsi_image_number, row, column))
                np_to_pil(np_tile_mask).save('{}/{}/{}_r{}c{}.png'.format(dir_to_save, "02-mask", wsi_image_number, row, column))
                pil_tile_roi.save('{}/{}/{}_r{}c{}.png'.format(dir_to_save, "03-roi", wsi_image_number, row, column))

                count_tiles_tumor += 1

#         logger.info("\t {} tumor regions.".format(cont_tumor_regions))
#         logger.info("\t {} tumor regions.".format(cont_tumor_regions))
    logger.info("\t {} patches of 640x640 size.".format(len(heat_grid_normal)))
    logger.info("\t\t {} patches of 640x640 (normal tissue).".format(count_tiles_normal))
    logger.info("\t\t {} patches of 640x640 (tumor).".format(count_tiles_tumor))
    logger.info("-")